In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Set up data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define directories for train, validation, and test sets
train_dir = 'D:/FolioList/Folio Leaf Dataset/Folio/'
validation_dir = 'D:/FolioList/Folio Leaf Dataset/Folio/'
test_dir = 'D:/FolioList/Folio Leaf Dataset/Folio/'

# Create train and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load DenseNet121 base model
base_model_densenet = DenseNet121(weights='imagenet', include_top=False)

# Add custom classification head with dropout
x = base_model_densenet.output
x = GlobalAveragePooling2D()(x) 
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout regularization
predictions_densenet = Dense(32, activation='softmax')(x)

# Combine base model with custom head
model_densenet = Model(inputs=base_model_densenet.input, outputs=predictions_densenet)

# Fine-tune some layers of the pre-trained DenseNet121 model
for layer in model_densenet.layers[:-10]:
    layer.trainable = False

# Compile the model
model_densenet.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model_densenet.fit(train_generator,
                             steps_per_epoch=train_generator.samples // train_generator.batch_size,
                             validation_data=validation_generator,
                             validation_steps=validation_generator.samples // validation_generator.batch_size,
                             epochs=10)

# Perform SVM and KNN classification
# Get the features and labels for train and validation data from the generators
X_train, y_train = train_generator.next()
X_val, y_val = validation_generator.next()

# Reshape the feature data to 2D array
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_val_flattened = X_val.reshape(X_val.shape[0], -1)

# Support Vector Machine (SVM)
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_flattened, y_train)
svm_val_predictions = svm_classifier.predict(X_val_flattened)
svm_accuracy = accuracy_score(y_val, svm_val_predictions)
print("SVM Validation Accuracy:", svm_accuracy)

# K-Nearest Neighbors (KNN)
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train_flattened, y_train)
knn_val_predictions = knn_classifier.predict(X_val_flattened)
knn_accuracy = accuracy_score(y_val, knn_val_predictions)
print("KNN Validation Accuracy:", knn_accuracy)

# Plotting accuracies
classifiers = ['DenseNet121', 'SVM', 'KNN']
accuracies = [history.history['val_accuracy'][-1], svm_accuracy, knn_accuracy]

plt.bar(classifiers, accuracies)
plt.xlabel('Classifiers')
plt.ylabel('Accuracy')
plt.title('Comparison of Classifier Accuracies')
plt.show()

       

Found 511 images belonging to 32 classes.
Found 126 images belonging to 32 classes.
Epoch 1/10


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 460s 20s/step - accuracy: 0.1386 - loss: 3.5032 - val_accuracy: 0.6667 - val_loss: 1.8680
Epoch 2/10
 1/15 ━━━━━━━━━━━━━━━━━━━━ 1:19 6s/step - accuracy: 0.4375 - loss: 1.9558

C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


15/15 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.4375 - loss: 1.0431 - val_accuracy: 0.6667 - val_loss: 0.8498
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 335s 14s/step - accuracy: 0.6366 - loss: 1.3552 - val_accuracy: 0.8125 - val_loss: 1.1744
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.7812 - loss: 0.4780 - val_accuracy: 0.8333 - val_loss: 0.5072
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 350s 15s/step - accuracy: 0.8300 - loss: 0.7141 - val_accuracy: 0.8125 - val_loss: 0.7109
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.8438 - loss: 0.2571 - val_accuracy: 0.7333 - val_loss: 0.3867
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 338s 15s/step - accuracy: 0.9061 - loss: 0.4022 - val_accuracy: 0.8750 - val_loss: 0.5238
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 22s 827ms/step - accuracy: 0.9062 - loss: 0.2119 - val_accuracy: 0.9333 - val_loss: 0.1568
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 313s 12s/step - accuracy: 0.9319 - loss: 0.2603 - val_accuracy: 0.8646 - val_loss:

AttributeError: 'DirectoryIterator' object has no attribute 'next'